In [1]:
import numpy as np

$$ AX=B $$

## Utility methods

In [2]:
def is_diagonally_dominant(A):
    for i in range(A.shape[0]):
        if 2 * abs(A[i,i]) < np.sum(np.abs(A[i,:])):
            return False
    return True

In [3]:
def random_diagonally_dominant(n):
    A = np.random.rand(n,n) * 2 - 1
    for i in range(n):
        A[i,i] = np.sum(np.abs(A[i,:]))
    return A

## Matrix methods
$$ A = L + D + U$$   
<center>
    $ D $ - diagonal 
    $ L $ - lower triangle 
    $ U $ - upper triangle
</center>

In [4]:
def matrix_L(A):
    return np.tril(A, -1)

def matrix_D(A):
    return np.diag(np.diag(A))

def matrix_U(A):
    return np.triu(A, 1)

$$ R = L+U $$

In [5]:
def matrix_R(A):
    return matrix_L(A) + matrix_U(A)

$$ S = (L + D)^{-1}$$

In [6]:
def matrix_S(A):
    return np.linalg.inv(matrix_L(A) + matrix_D(A))

$$ N=D^{-1} $$

In [7]:
def matrix_N(A):
    return np.linalg.inv(np.diag(np.diag(A)))

$$ C_1 = D^{-1}B = NB $$

In [8]:
def matrix_C1(A, B):
    return np.dot(matrix_N(A), B)

$$ T_1 = -D^{-1}(L + U) = -NR $$

In [9]:
def matrix_T1(A):
    return -1 * np.dot(matrix_N(A), matrix_R(A))

$$ C_2 = (L+D)^{-1}B = SB $$

In [10]:
def matrix_C2(A, B):
    return np.dot(matrix_S(A), B)

$$ T_2 = -(L+D)^{-1}U = -SU $$

In [11]:
def matrix_T2(A):
    return -1 * np.dot(matrix_S(A), matrix_U(A))

## Jacobi method
$$ X^{(k+1)} = T_1X^{(k)}+C_1 $$

In [12]:
def jacobi_compute(T1, C1, X):
    return np.dot(T1, X) + C1

In [13]:
def jacobi(A, B, X, iterations):
    if not is_diagonally_dominant(A):
        raise ValueError("Given matrix is not diagonally dominant")
    T1 = matrix_T1(A)
    C1 = matrix_C1(A, B)
    for i in range(iterations):
        X = jacobi_compute(T1, C1, X)
    return X

## Gauss-Seidel method
$$ X^{(k+1)} = T_2X^{(k)}+C_2 $$

In [14]:
def gauss_seidel_compute(T2, C2, X):
    return np.dot(T2, X) + C2

In [15]:
def gauss_seidel(A, B, X, iterations):
    if not is_diagonally_dominant(A):
        raise ValueError("Given matrix is not diagonally dominant")
    T2 = matrix_T2(A)
    C2 = matrix_C2(A, B)
    for i in range(iterations):
        X = gauss_seidel_compute(T2, C2, X)
    return X

---

#### Test 1

In [16]:
A = np.array([[ 2. , 1.],
              [ 5. , 7.]])

B = np.array([[11.],[13.]])
X = np.array([[1.],[1.]])

In [17]:
jacobi(A, B, X, 25)

array([[ 7.11110202],
       [-3.22220342]])

In [18]:
gauss_seidel(A, B, X, 25)

array([[ 7.11111111],
       [-3.22222222]])

In [19]:
np.linalg.solve(A, B)

array([[ 7.11111111],
       [-3.22222222]])

#### Test 2

In [20]:
A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0., 3., -1., 8.]])

B = np.array([[6.], [25.], [-11.], [15.]])
X = np.array([[0.],[0.],[0.],[0,]])

In [21]:
jacobi(A, B, X, 25)

array([[ 1.],
       [ 2.],
       [-1.],
       [ 1.]])

In [22]:
gauss_seidel(A, B, X, 25)

array([[ 1.],
       [ 2.],
       [-1.],
       [ 1.]])

In [23]:
np.linalg.solve(A, B)

array([[ 1.],
       [ 2.],
       [-1.],
       [ 1.]])

#### Test 3

In [24]:
A = np.array([[4., -1., -1.],
              [-2., 6., 1.],
              [-1., 1., 7.]])

B = np.array([[3.], [9.], [-6.]])
X = np.array([[0.],[0.],[0.]])

In [25]:
jacobi(A, B, X, 25)

array([[ 1.],
       [ 2.],
       [-1.]])

In [26]:
gauss_seidel(A, B, X, 25)

array([[ 1.],
       [ 2.],
       [-1.]])

In [27]:
np.linalg.solve(A, B)

array([[ 1.],
       [ 2.],
       [-1.]])

#### Test 4 - not diagonally dominant

In [28]:
A = np.array([[4., -1., -1.],
              [-2., 1., 1.],
              [-1., 1., 7.]])

B = np.array([[3.], [9.], [-6.]])
X = np.array([[0.],[0.],[0.]])

In [29]:
is_diagonally_dominant(A)

False

In [30]:
jacobi(A, B, X, 25)

array([[ 5.9710198 ],
       [24.56950449],
       [-3.48213643]])

In [31]:
gauss_seidel(A, B, X, 25)

array([[ 5.99999762],
       [24.49999439],
       [-3.49999954]])

In [32]:
np.linalg.solve(A, B)

array([[ 6. ],
       [24.5],
       [-3.5]])

---

In [33]:
def test(n):
    A = random_diagonally_dominant(n)
    B = np.random.rand(n,1)
    T1 = matrix_T1(A)
    T2 = matrix_T2(A)
    C1 = matrix_C1(A, B)
    C2 = matrix_C2(A, B)
    Xj = np.zeros((n,1))
    Xg = np.zeros((n,1))
    X = np.linalg.solve(A, B)
    
    data = list()
    
    while (not np.allclose(Xj, X)) or (not np.allclose(Xg, X)):
        Xj = jacobi_compute(T1, C1, Xj)
        Xg = gauss_seidel_compute(T2, C2, Xg)
        
        print(str(np.sum(np.abs(X - Xj))) + "," + str(np.sum(np.abs(X - Xg))))

In [34]:
%%capture --no-stderr --no-display log3
test(3)

In [35]:
with open('results3.csv', 'w') as f:
    f.write("Jacobi,Gauss-Seidel\n")
    f.write(log3.stdout)

In [36]:
%%capture --no-stderr --no-display log5
test(5)

In [37]:
with open('results5.csv', 'w') as f:
    f.write("Jacobi,Gauss-Seidel\n")
    f.write(log5.stdout)

In [38]:
%%capture --no-stderr --no-display log10
test(10)

In [39]:
with open('results10.csv', 'w') as f:
    f.write("Jacobi,Gauss-Seidel\n")
    f.write(log10.stdout)

In [40]:
%%capture --no-stderr --no-display log30
test(30)

In [41]:
with open('results30.csv', 'w') as f:
    f.write("Jacobi,Gauss-Seidel\n")
    f.write(log30.stdout)

In [42]:
%%capture --no-stderr --no-display log100
test(100)

In [43]:
with open('results100.csv', 'w') as f:
    f.write("Jacobi,Gauss-Seidel\n")
    f.write(log100.stdout)

In [44]:
%%capture --no-stderr --no-display log1000
test(1000)

In [45]:
with open('results1000.csv', 'w') as f:
    f.write("Jacobi,Gauss-Seidel\n")
    f.write(log1000.stdout)